# Only for Colab

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/Colab Notebooks/2019NN/

/content/gdrive/My Drive/Colab Notebooks/2019NN


In [1]:
!ls

data  data_helpers  Train_Hotel.ipynb  Train_Hotel.py


In [6]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


# Start

In [1]:
from fastai.text import *
from data_helpers.Data import *
from fastai.text.transform import *

In [2]:
sent_num_file = ["aspect_0.count", "test_aspect_0.count"]
rating_file = ["aspect_0.rating", "test_aspect_0.rating"]
content_file = ["aspect_0.txt", "test_aspect_0.txt"]

dataset_dir = "./data/hotel_balance_LengthFix1_3000per/"

In [3]:
def concat_to_doc(sent_list, sent_count):
    start_index = 0
    docs = []
    for s in sent_count:
        doc = " xxPERIOD ".join(sent_list[start_index:start_index + s])
        doc = doc + " xxPERIOD "
        docs.append(doc)
        start_index = start_index + s
    return docs

In [4]:
TRAIN_DATA = 0
TEST_DATA = 1

## Load Training Data

In [5]:
# Load Count
sent_count_train = list(open(dataset_dir + sent_num_file[TRAIN_DATA], "r").readlines())
sent_count_train = [int(s) for s in sent_count_train if (len(s) > 0 and s != "\n")]
print( sent_count_train[0:5] )

# Load Ratings
aspect_rating_train = list(open(dataset_dir + rating_file[TRAIN_DATA], "r").readlines())
aspect_rating_train = [s for s in aspect_rating_train if (len(s) > 0 and s != "\n")]

aspect_rating_train = [s.split(" ") for s in aspect_rating_train]
aspect_rating_train = np.array(aspect_rating_train)[:, 0:-1]
aspect_rating_train = aspect_rating_train.astype(np.int) - 1
aspect_rating_train = pd.DataFrame(aspect_rating_train)
print( aspect_rating_train.head() )

# Load Sents
sents_train = list(open(dataset_dir + content_file[TRAIN_DATA], "r").readlines())
sents_train = [s.strip() for s in sents_train]
print( sents_train[0:5] )

[10, 14, 14, 24, 23]
   0  1  2  3  4  5
0  4  4  4  4  4  4
1  4  3  4  4  4  4
2  1  0  1  1  2  1
3  3  2  3  2  4  3
4  3  2  1  2  2  4
['we had the most wonderful time at the excellence', 'we opted for the excellence club and it was well worht it for the perks', 'roberto puello who was the club manager did an extra special job of making us feel welcomed', 'we took some great pictures that looked like postcards', 'the food was excellent the pizza was close to being our favorite']


In [6]:
docs_train = concat_to_doc(sents_train, sent_count_train)

docs_train = pd.DataFrame(docs_train)

docs_train.head()

,0
0,we had the most wonderful time at the excellen...
1,a truly fantastic place xmas we have just ret...
2,too sick to enjoy the ride to and form the air...
3,"relaxing and fun , but xxPERIOD i rang in the ..."
4,somewhat excellence we just returned from a on...


In [7]:
docs_train.iloc[0][0]

"we had the most wonderful time at the excellence xxPERIOD we opted for the excellence club and it was well worht it for the perks xxPERIOD roberto puello who was the club manager did an extra special job of making us feel welcomed xxPERIOD we took some great pictures that looked like postcards xxPERIOD the food was excellent the pizza was close to being our favorite xxPERIOD the shows were great especially the michael jackson and the 70's  80's night xxPERIOD the french resturant was probably our favorite xxPERIOD we will definately be back for an anniversary trip xxPERIOD we made some friends but had plenty of privcacy as well xxPERIOD the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well xxPERIOD "

In [8]:
df_train = pd.concat( [aspect_rating_train, docs_train], axis=1, ignore_index=True )
df_train.head()

,0,1,2,3,4,5,6
0,4,4,4,4,4,4,we had the most wonderful time at the excellen...
1,4,3,4,4,4,4,a truly fantastic place xmas we have just ret...
2,1,0,1,1,2,1,too sick to enjoy the ride to and form the air...
3,3,2,3,2,4,3,"relaxing and fun , but xxPERIOD i rang in the ..."
4,3,2,1,2,2,4,somewhat excellence we just returned from a on...


In [9]:
df_train.shape

(11173, 7)

## Load Testing Data

In [10]:
# Load Count
sent_count_test = list(open(dataset_dir + sent_num_file[TEST_DATA], "r").readlines())
sent_count_test = [int(s) for s in sent_count_test if (len(s) > 0 and s != "\n")]
print( sent_count_test[0:5] )

# Load Ratings
aspect_rating_test = list(open(dataset_dir + rating_file[TEST_DATA], "r").readlines())
aspect_rating_test = [s for s in aspect_rating_test if (len(s) > 0 and s != "\n")]

aspect_rating_test = [s.split(" ") for s in aspect_rating_test]
aspect_rating_test = np.array(aspect_rating_test)[:, 0:-1]
aspect_rating_test = aspect_rating_test.astype(np.int) - 1
aspect_rating_test = pd.DataFrame(aspect_rating_test)
print( aspect_rating_test.head() )

# Load Sents
sents_test = list(open(dataset_dir + content_file[TEST_DATA], "r").readlines())
sents_test = [s.strip() for s in sents_test]

# Sents to Doc
docs_test = concat_to_doc(sents_test, sent_count_test)

docs_test = pd.DataFrame(docs_test)
print( docs_test.head() )

[9, 30, 25, 33, 29]
   0  1  2  3  4  5
0  1  0  0  3  1  1
1  2  2  1  2  3  3
2  4  4  4  3  4  4
3  3  2  3  3  3  4
4  3  4  3  4  4  4
                                                   0
0  definitely not a 5 star resort i 'm dumbfounde...
1  facilities need work xxPERIOD we visited excel...
2  excellence was exactly that xxPERIOD my family...
3  great service , nice hotel , mediocre food xxP...
4  very relaxing experience just returned from my...


In [11]:
docs_test.iloc[0][0]

"definitely not a 5 star resort i 'm dumbfounded that this hotel gets good reviews and is so highly rated xxPERIOD it 's decidedly a 3 star property , not 5 stars as indicated xxPERIOD the rooms are very dated and run down , old crappy beds and pillows , an old tv and overall poorly maintained xxPERIOD the whole property is pretty run down and old - looking xxPERIOD the food is subpar , not one meal i had would be called great xxPERIOD the service is uneven and the staff is poorly trained and uninformed xxPERIOD many do not comprehend english xxPERIOD the beach is great , it 's the only redeeming factor xxPERIOD however the resort is a 1- hour taxi trip from the airport xxPERIOD "

In [12]:
df_test = pd.concat( [aspect_rating_test, docs_test], axis=1, ignore_index=True )
df_test.head()

,0,1,2,3,4,5,6
0,1,0,0,3,1,1,definitely not a 5 star resort i 'm dumbfounde...
1,2,2,1,2,3,3,facilities need work xxPERIOD we visited excel...
2,4,4,4,3,4,4,excellence was exactly that xxPERIOD my family...
3,3,2,3,3,3,4,"great service , nice hotel , mediocre food xxP..."
4,3,4,3,4,4,4,very relaxing experience just returned from my...


In [14]:
df_test.shape

(3739, 7)

# Build Language Model Data Bunch

In [29]:
lmdb = TextLMDataBunch.from_df("./data/", train_df=df_train, valid_df=df_test, text_cols=6, label_cols=[0,1,2,3,4,5])

In [30]:
len( lmdb.vocab.itos )

23008

In [14]:
lmdb.vocab.itos[0:8]

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxmaj', 'xxup', 'xxrep']

In [15]:
lmdb.show_batch()

idx,text
0,"were great especially the michael jackson and the 70 's 80 's night xxperiod the french resturant was probably our favorite xxperiod we will definately be back for an anniversary trip xxperiod we made some friends but had plenty of xxunk as well xxperiod the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well xxperiod xxbos a"
1,xxperiod if you 're craving the coconut meat inside the coconut ( which was tasty ) then ask the bartender to crack open the coconut after you ' ve finished your drink xxperiod the lobster house was * hot * and if it 's been a hot day i would not recommend eating there xxperiod also we found the steamed lobster to be much better than the grilled ( we
2,"advance to arrange a taxi , driver was waiting with a sign xxperiod xxunk five minutes , sixty bucks and a ten dollar tip later arrived at excellence xxperiod the drive nothing to worry about , xxunk through the potholes , application of the horn as an occasional xxunk aid , all fairly standard in our experience xxperiod you never really get going all that fast xxperiod certainly less of"
3,"check more prices at the vendors nextdoor they will sell it xxperiod those guys are some competition with eachother xxperiod they put towels out each hour , around 3 pm they disappear but you can find them at the clubhouse ( on beach its a mini gift shop / where you meet for the horse ride ) or at the pool behind the bar there is a hut that they"
4,might have been some xxunk with the hotel but that is to be expected as any place ages xxperiod the ride from the airport was about an hour in a mini van which was included in our package xxperiod our driver was great he stopped along the way so the guys could grab a couple of beers xxperiod our check - in was a breeze and the bellboys were very


In [16]:
moms = (0.8,0.7)

In [17]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,4.000441,3.756649,0.287865,01:57
1,3.788871,3.586668,0.304594,02:02
2,3.631943,3.490414,0.315590,01:59
3,3.458348,3.431321,0.322942,01:59
4,3.387737,3.421412,0.324441,02:00


In [24]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,3.961397,3.744603,0.289252,06:04
1,3.766985,3.583916,0.304960,06:13
2,3.601011,3.487880,0.315974,06:14
3,3.475492,3.431211,0.322712,06:15
4,3.398683,3.420531,0.324421,06:15


In [18]:
lmdb.save('lm_databunch_hotel.1115')

In [19]:
learn.save('lm_learner_hotel.1115')

In [25]:
learn.save_encoder('lm_enc_hotel.1115')

# Build Classifier Data Bunch

In [38]:
np.random.seed(42)
msk = np.random.rand(len(df_train)) < 0.8
df_train08 = df_train[msk]
df_valid02 = df_train[~msk]
len(df_train08), len(df_valid02)

(8969, 2204)

In [39]:
clas_db_tvt = TextClasDataBunch.from_df("./data/",
                                     train_df=df_train08,
                                     valid_df=df_valid02,
                                     test_df=df_test,
                                     text_cols=6,
                                     label_cols=[0,1,2,3,4,5],
                                     vocab=lmdb.train_ds.vocab,
                                     bs=42)

In [45]:
clas_db_tv = TextClasDataBunch.from_df("./data/",
                                     train_df=df_train,
                                     valid_df=df_test,
                                     text_cols=6,
                                     label_cols=[0,1,2,3,4,5],
                                     vocab=lmdb.train_ds.vocab,
                                     bs=42)

In [40]:
clas_db.show_batch()

text,target
"xxbos "" an extremely disappointing experience "" where do i beging xxperiod first let me just say that originally i had booked a reservation to stay at this hotel for 9 days instead of the two days that i lasted with my family at this hotel xxperiod the original comfirmed reservation was to have taken place from friday august 15 to sunday august xxunk in may this year i was",
"xxbos warning to all - read before going xxperiod we went to the bahia from may 4 - 11 xxperiod i felt it important to share my experience with all of you as i did read posts from this site prior to booking xxperiod day 1 friday : arrived at hotel , when you get off the bus , the hotel tags your luggage and they tell you to go",
"xxbos we had a fabulous stay xxperiod we stayed at the edenh from march 8 ? xxperiod our stay was beyond fantastic and our only complaint is that by this time next year we won ? ? be able to afford to stay there again xxperiod we arrived at the hotel at approximately 5:30pm , our check in was smooth with absolutely no problems , the bell boy took our",
"xxbos i had a free room two things , first xxperiod one , i ' m picky xxperiod two , this stay was free for me as a friend was attending a conference and her husband could not join her - - i was asked if i 'd like to come along and well , i work for an airline and get nearly free airfare and so , what could",2;4
"xxbos interesting my husband , three year old and i just returned from punta cana tonight xxperiod so ends an interesting week xxperiod we flew to the dominican , for our 2nd visit , on valentine 's day xxperiod we had a private vehicle pick us up at the airport and arrived quite quickly xxperiod check - in was quick and we were given a very nice room xxperiod the",


In [33]:
x,y = clas_db.one_batch()
y[0:5]

tensor([[4., 3., 3., 3., 4., 4.],
        [2., 4., 3., 4., 4., 1.],
        [2., 4., 0., 3., 3., 0.],
        [3., 2., 0., 3., 1., 2.],
        [1., 2., 1., 3., 3., 4.]])

In [23]:
period_index = clas_db.vocab.stoi["xxperiod"]
period_index

10

In [32]:
x[0].cpu()[0:50] == period_index

tensor([False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False, False])

In [34]:
clas_db.vocab.textify(x.cpu()[0,0:50])

'xxbos better than expected with a toddler too xxperiod background about us : we are a traveling type of family from humid houston , texas and have been to numerous carribean resorts , this was our first time to the dr , punta cana xxperiod please note that we are'

In [41]:
clas_db_tvt.save("hotel_clas_databunch.TraValTes")

In [46]:
clas_db_tv.save("hotel_clas_databunch.TraVal")

##  -- load LM Databunch and LM Learner

In [4]:
lmdb = load_data("./data/", "hotel_lm_databunch.1001")

In [5]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn = learn.load("lang_model_hotel")

freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


## -- load databunch

In [6]:
clas_db = load_data("./data/", "hotel_cls_databunch.aspect_only")

In [7]:
clas_db.batch_size=2

In [8]:
clas_db.c

5

In [14]:
cls_learn = text_classifier_learner(clas_db, AWD_LSTM)

freezing:Embedding(23008, 400, padding_idx=1)
freezing:EmbeddingDropout(
  (emb): Embedding(23008, 400, padding_idx=1)
)
freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


# Model Test

In [8]:
encoder = learn.model[0]
encoder.reset()

In [9]:
learn.predict("the rooms are very", n_words=20)

'the rooms are very prodding with this xxperiod we got this but we were excited about going to melia tropical due to the fact'